# Setup

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.2.0'

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [5]:
items = pd.read_csv("../Datasets/ml-100k/Text/items.csv")

In [6]:
data_text = items['Summary']

In [7]:
data_text[0]

'A little boy named Andy loves to be in his room, playing with his toys, especially his doll named "Woody". But, what do the toys do when Andy is not with them, they come to life. Woody believes that his life (as a toy) is good. However, he must worry about Andy\'s family moving, and what Woody does not know is about Andy\'s birthday party. Woody does not realize that Andy\'s mother gave him an action figure known as Buzz Lightyear, who does not believe that he is a toy, and quickly becomes Andy\'s new favorite toy. Woody, who is now consumed with jealousy, tries to get rid of Buzz. Then, both Woody and Buzz are now lost. They must find a way to get back to Andy before he moves without them, but they will have to pass through a ruthless toy killer, Sid Phillips. —John Wiggins'

# AutoEncoder
## Text Data

In [8]:
import re

from keras.optimizers import Adam
from keras.models import Model, Sequential

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import wordnet
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from nltk.tokenize import word_tokenize as wt

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.losses import sparse_categorical_crossentropy
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Bidirectional

Using TensorFlow backend.
[nltk_data] Downloading package punkt to /home/sriram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sriram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sriram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
lem = wordnet.WordNetLemmatizer()
sw = set(stopwords.words("english")) 

In [10]:
# self.input_sequence = Input(shape = (self.max_len,))
# self.embedding = Embedding(input_dim = self.vocab, output_dim=128,)(self.input_sequence)
# self.encoder = Bidirectional(LSTM(self.hidden, return_sequences=False))(self.embedding)
# self.r_vec = RepeatVector(self.max_len)(self.encoder)
# self.decoder = Bidirectional(LSTM(self.hidden, return_sequences=True, dropout=0.2))(self.r_vec)
# self.logits = TimeDistributed(Dense(self.vocab))(self.decoder)
# self.enc_dec_model = Model(inputs = self.input_sequence, outputs = Activation('softmax')(self.logits))
# enc_inp = Input(shape = (self.max_len,))

In [11]:
class Autoencoder_Text(Model):
    def __init__(self, input_=300, hidden=50):
        super(Autoencoder_Text, self).__init__()
        self.max_len = input_
        self.hidden = hidden
    
    def clean_text(self, text):
        text = str(text).lower()
        text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
        text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
        text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
        text = re.sub('\t', ' ',  text)
        text = re.sub(r" +", ' ', text)
        text = wt(text)
        text = (" ").join([lem.lemmatize(i, pos ='v') 
                           for i in text if i not in sw])
        return(text)
    
    def tokenize(self, sentences):
        text_tokenizer = Tokenizer()
        text_tokenizer.fit_on_texts(sentences)
        return(text_tokenizer.texts_to_sequences(sentences), text_tokenizer)
    
    def pre_process(self, data_text, train=False):
        data_text = data_text.apply(self.clean_text)
        text_tokenized, self.text_tokenizer = self.tokenize(data_text)
        
        if train:
            self.vocab = len(self.text_tokenizer.word_index) + 1
            self.max_len = int(len(max(text_tokenized,key=len)))
            
        pad_sentence = pad_sequences(text_tokenized, self.max_len, padding = "post")
        return(pad_sentence.reshape(*pad_sentence.shape, 1))
    
    def model(self):
        self.encoder = Sequential([
            Embedding(input_dim = self.vocab, output_dim=128,),
            Bidirectional(LSTM(self.hidden, return_sequences=False))
        ])
        
        self.r_vec = RepeatVector(self.max_len)
        self.decoder = Sequential([
            Bidirectional(LSTM(self.hidden, return_sequences=True, dropout=0.2)),
            TimeDistributed(Dense(self.vocab))
        ])
        self.enc_dec_model = Sequential([self.encoder, self.r_vec, self.decoder])
    
    def train(self, data_text, epochs=10, batch_size=20):
        pad_sentence = self.pre_process(data_text, True)
        self.model()
        self.enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
        
        self.enc_dec_model.summary()
        self.enc_dec_model.fit(np.squeeze(pad_sentence, axis = 2), 
                                      pad_sentence, batch_size=batch_size, epochs=epochs)
        self.enc_dec_model.save('./pretrained/text_model')
        
    def call(self, inputs):
        pad_sentences = self.pre_process(inputs, False)
        print(tf.convert_to_tensor(pad_sentences).shape)
        return(self.enc_dec_model(tf.squeeze(pad_sentences, axis = 2)))

In [16]:
AE = Autoencoder_Text()
AE.train(data_text, epochs = 10, batch_size = 30)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_4 (Sequential)    (None, 100)               1821616   
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 240, 100)          0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 240, 13672)        1441272   
Total params: 3,262,888
Trainable params: 3,262,888
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1682/1682 [==============================] - 37s 22ms/step - loss: 3.2861 - accuracy: 0.7713
Epoch 2/10
1682/1682 [==============================] - 40s 24ms/step - loss: 2.5532 - accuracy: 0.7894
Epoch 3/10
1682/1682 [==============================] - 46s 27ms/step - loss: 2.6254 - accuracy: 0.7886
Epoch 4/10
1682/1682 [==============================] - 49s 29

In [17]:
inputs = np.squeeze(AE.pre_process(data_text), axis = 2)
output = AE.encoder.predict(inputs)
print(output[:2])

[[ 0.00580708  0.00328717 -0.06211844  0.04372958  0.01164463 -0.01974315
  -0.01029312  0.0251993  -0.03299069  0.00252017 -0.02111654  0.02876656
  -0.05950402  0.00679381  0.0250336   0.00219231 -0.001321   -0.10219908
  -0.01739231 -0.0861205   0.00977161 -0.01802992  0.00991577  0.01871123
  -0.04014775 -0.01935747  0.00491042  0.02275256  0.01923463  0.03762147
  -0.01565715 -0.0399536  -0.00464426  0.02601043 -0.04974602 -0.00426659
   0.01237303 -0.04691805 -0.01704604 -0.00839573  0.03441707 -0.02830616
  -0.03396834  0.02059674  0.03532096  0.01276242 -0.05463283  0.00992401
   0.03215405  0.00380808  0.03851245 -0.01175375 -0.03278746 -0.03338955
  -0.00962947 -0.01989641 -0.00212314 -0.06176991  0.00432015  0.01511423
  -0.00444017 -0.02132732  0.00099115  0.04299232 -0.00759095  0.03637276
  -0.00100477 -0.00238583 -0.00583881 -0.00277562  0.01362262  0.01131153
   0.01879097  0.00358059  0.02546507 -0.04044672  0.01184742  0.01985254
  -0.02415028  0.01888555  0.01400695 

## Meta-Data